# Reporte de uso de MP para gestion de reposiscion

In [8]:
import MySQLdb
from datetime import datetime, timedelta
import pandas as pd

db = MySQLdb.connect(host="192.168.0.12",    # tu host, usualmente localhost
                     user="operario",         # tu usuario
                     passwd="",  # tu password
                     db="dbp8100")        # el nombre de la base de datos

# Debes crear un objeto Cursor. Te permitirá
# ejecutar todos los queries que necesitas
cur = db.cursor()

In [9]:
fecha_actual = datetime.now()
fecha_actual = fecha_actual.strftime("%Y-%m-%d")

In [10]:
fecha_ayer = datetime.now()
fecha_ayer = fecha_ayer.strftime('%Y-%m-%d')

In [20]:
fecha_actual = input("Ingrese la fecha actual (YYYY-MM-DD): ") or fecha_actual
fecha_ayer = input("Ingrese la fecha de ayer (YYYY-MM-DD): ") or fecha_ayer

In [21]:
cur.execute("""
    SELECT 
        productox.Codigo AS Codigo, 
        productox.Nombre AS Producto, 
        SUM(dcaptura.Valor) AS Dosificado 
    FROM 
        dbp8100.dcaptura AS dcaptura
    JOIN 
        dbp8100.tareaseje AS tareaseje ON dcaptura.IDT = tareaseje.NroID
    JOIN 
        dbp8100.productox AS productox ON productox.NroID = dcaptura.IDP 
    WHERE 
        (tareaseje.Fecha = %s AND tareaseje.Hora >= '23:00:00') OR 
        (tareaseje.Fecha > %s AND tareaseje.Fecha <= %s AND tareaseje.Hora < '23:00:00')
    GROUP BY 
        productox.Nombre
""", (fecha_ayer, fecha_ayer, fecha_actual))

12

In [22]:
# Obtener los resultados
resultados = cur.fetchall()
# Obtener los nombres de las columnas
columnas = [desc[0] for desc in cur.description]

# Crear un DataFrame de pandas
df = pd.DataFrame(resultados, columns=columnas)
df['Dosificado'] = df['Dosificado'] * -1  # Convertir a negativo

In [23]:
df

,Codigo,Producto,Dosificado
0,B-07,AFRECHO ARROZ,-1343.7
1,C-12,CARBONATO DE CALCIO,-80.7
2,DESPT,DESPARASITAN-T,-1.8
3,A-03,EXP SOJA,-21624.0
4,C-19,MAGNUM 100,-48.8
5,A-02,MAIZ,-25458.0
6,C-19,MAXIMIX CERDAS REP PLUS 3.1,-68.5
7,A-08,PELLET AFRECHILLO DE TRIGO,-2300.0
8,C-11,SAL,-79.5
9,C-19,STARFEED DESARROLLO,-238.4
